In [6]:
import sys

sys.path.insert(0, "..")

import scipy.io
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from HAR.transformers import CSIScaler, Rocket
from HAR.classifier import RidgeVotingClassifier


def load_dataset(infile):
    mat = scipy.io.loadmat(infile)
    X = mat["csi"].T
    nsamples = mat["nsamples"].flatten()
    dim = mat["dim"].flatten()
    classnames = list(map(lambda s: s.strip().title(), mat["classnames"]))
    y = []
    for i in range(len(classnames)):
        y += [i] * nsamples[i]
    y = np.array(y)
    return X, y, nsamples, classnames, dim


pipe = Pipeline(
    [
        ("scaler", CSIScaler()),
        (
            "feature_selector",
            Rocket(n_kernels=1_500),
        ),
        (
            "classifier",
            RidgeVotingClassifier(n_classes=3),
        ),
    ]
)

In [7]:
X, y, nsamples, classnames, dim = load_dataset("../dataset/rCSI-3")
X = X.reshape(X.shape[0], *dim)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=y)

pipe.fit_transform(X_train, y_train)

y_pred = pipe.predict(X_test)

print(f"\n> Test Accuracy: {accuracy_score(y_test, y_pred)*100:.4f}%")
print("\n> Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))
print("\n> Classification Report :")
print(classification_report(y_test, y_pred, target_names=classnames))

=== Generating Kernels ===
=== Kernel Transform ===


100%|██████████| 1200/1200 [01:34<00:00, 12.73it/s]


=== Classifer Training ===


100%|██████████| 54/54 [00:21<00:00,  2.54it/s]


=== Kernel Transform ===


100%|██████████| 300/300 [00:28<00:00, 10.41it/s]



> Test Accuracy: 97.0000%

> Confusion Matrix: 
[[100   0   0]
 [  0  99   1]
 [  3   5  92]]

> Classification Report :
              precision    recall  f1-score   support

       Empty       0.97      1.00      0.99       100
        Walk       0.95      0.99      0.97       100
        Jump       0.99      0.92      0.95       100

    accuracy                           0.97       300
   macro avg       0.97      0.97      0.97       300
weighted avg       0.97      0.97      0.97       300



In [8]:
import warnings

warnings.filterwarnings("ignore")

X, y, nsamples, classnames, dim = load_dataset("../dataset/rCSI-4")
X = X.reshape(X.shape[0], *dim)

y_pred = pipe.predict(X)

print(f"\n> Test Accuracy: {accuracy_score(y, y_pred)*100:.4f}%")
print("\n> Confusion Matrix: ")
print(confusion_matrix(y, y_pred))
print("\n> Classification Report :")
print(classification_report(y, y_pred, target_names=classnames))

=== Kernel Transform ===


100%|██████████| 360/360 [00:28<00:00, 12.64it/s]



> Test Accuracy: 91.1111%

> Confusion Matrix: 
[[106   0  14]
 [  0 115   5]
 [  0  13 107]]

> Classification Report :
              precision    recall  f1-score   support

       Empty       1.00      0.88      0.94       120
        Walk       0.90      0.96      0.93       120
        Jump       0.85      0.89      0.87       120

    accuracy                           0.91       360
   macro avg       0.92      0.91      0.91       360
weighted avg       0.92      0.91      0.91       360

